In [1]:
pip install xml.etree.ElementTree

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement xml.etree.ElementTree (from versions: none)
ERROR: No matching distribution found for xml.etree.ElementTree

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\wpark11\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install requests

## DICOM Standard Part 6

In [1]:
import requests
import xml.etree.ElementTree as ET

# URI for DICOM Standard Part 6
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part06/part06.xml'

# Parse the XML content
response = requests.get(xml_uri)
root = ET.fromstring(response.content)

In [2]:
# Find the child node with label="6"
for child in root:
    if child.attrib.get('label') == '6':
        selected_node = child
        break

In [3]:
import pandas as pd
from collections import defaultdict

# Get table 1 from Chapter 6
if selected_node is not None:
    grandchild_table = selected_node.find('.//{http://docbook.org/ns/docbook}table')
    if grandchild_table is not None:
        thead = grandchild_table.find('.//{http://docbook.org/ns/docbook}thead')
        tbody = grandchild_table.find('.//{http://docbook.org/ns/docbook}tbody')

        # Store column names
        column_names = []
        if thead is not None:
            for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                thead_names = tr.findall('.//{http://docbook.org/ns/docbook}emphasis')
                if thead_names is not None:
                    for name in thead_names:
                        column_names.append(name.text.strip())

            # Store row values
            rows = tbody.findall('.//{http://docbook.org/ns/docbook}tr')
            rows_data = []

            # Loop through tbody to extract values
            for tr in rows:
                row_values = defaultdict(lambda: None)
                idx = 0
                for para in tr.findall('.//{http://docbook.org/ns/docbook}para'):
                    emphasis = para.find('.//{http://docbook.org/ns/docbook}emphasis')
                    if emphasis is not None and emphasis.text is not None:
                        row_values[column_names[idx]] = emphasis.text.strip()
                        idx += 1
                        if idx >= len(column_names):
                            break
                    else:
                        if para is not None and para.text is not None:
                            row_values[column_names[idx]] = para.text.strip()
                            idx += 1
                            if idx >= len(column_names):
                                break

                # Append to the table only if it has more than 3 values
                if sum(1 for value in row_values.values() if value is not None) > 3:
                    rows_data.append(row_values)

            # Save the output as a DataFrame
            df = pd.DataFrame(rows_data, columns=column_names)
else:
    print("Node with label='6' not found.")


In [4]:
column_names

['Tag', 'Name', 'Keyword', 'VR', 'VM']

In [5]:
df.head(10)

,Tag,Name,Keyword,VR,VM
0,"(0008,0001)",Length to End,Length​To​End,UL,1
1,"(0008,0005)",Specific Character Set,Specific​Character​Set,CS,1-n
2,"(0008,0006)",Language Code Sequence,Language​Code​Sequence,SQ,1
3,"(0008,0008)",Image Type,Image​Type,CS,2-n
4,"(0008,0010)",Recognition Code,Recognition​Code,SH,1
5,"(0008,0012)",Instance Creation Date,Instance​Creation​Date,DA,1
6,"(0008,0013)",Instance Creation Time,Instance​Creation​Time,TM,1
7,"(0008,0014)",Instance Creator UID,Instance​Creator​UID,UI,1
8,"(0008,0015)",Instance Coercion DateTime,Instance​Coercion​Date​Time,DT,1
9,"(0008,0016)",SOP Class UID,SOP​Class​UID,UI,1


In [20]:
included_VR = ['AT', 'CS', 'DA', 'DT', 'DS', 'FL', 'FD', 'IS', 'SL', 'SS', 'SV', 'TM', 'UL', 'US', 'UV']
df[df['VR'].isin(included_VR)].VR.value_counts() #CS is values; others are numbers 

VR
CS    799
DS    483
US    379
FD    309
FL    292
IS    239
UL     79
DT     62
DA     61
TM     55
AT     24
SS     18
SL     15
UV      6
SV      1
Name: count, dtype: int64

## DICOM Standard Part 16

In [6]:
import requests
import xml.etree.ElementTree as ET

# URI for DICOM Standard Part 6
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part16/part16.xml'

# Parse the XML content
response = requests.get(xml_uri)
root = ET.fromstring(response.content)

In [7]:
# Find the child node with label="6"
for child in root:
    if child.attrib.get('label') == 'B':
        selected_node = child         
        break
print(selected_node.attrib)

{'label': 'B', 'status': '1', '{http://www.w3.org/XML/1998/namespace}id': 'chapter_B'}


In [6]:
len(selected_node.findall('.//{http://docbook.org/ns/docbook}table'))

1347

In [8]:
import pandas as pd

# Get all tables from Chapter B
if selected_node is not None:
    table_elements = selected_node.findall('.//{http://docbook.org/ns/docbook}table')

    # Dictionary to store DataFrames
    tables_dict = {}

    # Iterate through each table
    for index, table_element in enumerate(table_elements):
        thead = table_element.find('.//{http://docbook.org/ns/docbook}thead')
        tbody = table_element.find('.//{http://docbook.org/ns/docbook}tbody')

        # Store column names
        column_names = []
        if thead is not None:
            # Store the column names for the first row
            for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                for idx in range(len(tr)):
                    if tr[idx][0].text is not None:
                        column_names.append(tr[idx][0].text.strip())
        # print(f"Column names for Table {index + 1}: {column_names}")


        # Store row values
        rows_data = []
        if tbody is not None:
            # Iterate through each row in tbody
            for tr in tbody.findall('.//{http://docbook.org/ns/docbook}tr'):
                row_values = {}
                for idx, td in enumerate(tr.findall('.//{http://docbook.org/ns/docbook}td')):
                    if idx < len(column_names):
                        # Extract values from nested elements, such as links
                        value = ' '.join(text.strip() for text in td.itertext())
                        if value:
                            row_values[column_names[idx]] = value.strip()

                rows_data.append(row_values)

        # Save the output as a DataFrame and add to the dictionary
        df = pd.DataFrame(rows_data, columns=column_names)
        tables_dict[f'df_{index + 1}'] = df

else:
    print("Node with label='B' not found.")


## As functions

In [9]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict

def extract_DICOM_Part_6_table_data(xml_url, label):
    # Parse the XML content
    response = requests.get(xml_url)
    root = ET.fromstring(response.content)

    # Find the child node with the specified label
    selected_node = None
    for child in root:
        if child.attrib.get('label') == label:
            selected_node = child
            break

    # Get the first table from the selected node
    if selected_node is not None:
        grandchild_table = selected_node.find('.//{http://docbook.org/ns/docbook}table')
        if grandchild_table is not None:
            thead = grandchild_table.find('.//{http://docbook.org/ns/docbook}thead')
            tbody = grandchild_table.find('.//{http://docbook.org/ns/docbook}tbody')

            # Store column names
            column_names = []
            if thead is not None:
                for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                    thead_names = tr.findall('.//{http://docbook.org/ns/docbook}emphasis')
                    if thead_names is not None:
                        for name in thead_names:
                            column_names.append(name.text.strip())

            # Store row values
            rows_data = []
            if tbody is not None:
                # Loop through tbody to extract values
                for tr in tbody.findall('.//{http://docbook.org/ns/docbook}tr'):
                    row_values = defaultdict(lambda: None)
                    idx = 0
                    for para in tr.findall('.//{http://docbook.org/ns/docbook}para'):
                        emphasis = para.find('.//{http://docbook.org/ns/docbook}emphasis')
                        if emphasis is not None and emphasis.text is not None:
                            row_values[column_names[idx]] = emphasis.text.strip()
                            idx += 1
                            if idx >= len(column_names):
                                break
                        else:
                            if para is not None and para.text is not None:
                                row_values[column_names[idx]] = para.text.strip()
                                idx += 1
                                if idx >= len(column_names):
                                    break

                    # Append to the table only if it has more than 3 values
                    if sum(1 for value in row_values.values() if value is not None) > 3:
                        rows_data.append(row_values)

                # Save the output as a DataFrame
                df = pd.DataFrame(rows_data, columns=column_names)

        else:
            print(f"No table found in node with label='{label}'.")

    else:
        print(f"Node with label='{label}' not found.")


In [27]:
# URI for DICOM Standard Part 6
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part06/part06.xml'

# Extract tables with label '6'
extract_DICOM_Part_6_table_data(xml_uri, '6')

In [38]:
df.head(5)

,Tag,Name,Keyword,VR,VM
0,"(0008,0001)",Length to End,Length​To​End,UL,1
1,"(0008,0005)",Specific Character Set,Specific​Character​Set,CS,1-n
2,"(0008,0006)",Language Code Sequence,Language​Code​Sequence,SQ,1
3,"(0008,0008)",Image Type,Image​Type,CS,2-n
4,"(0008,0010)",Recognition Code,Recognition​Code,SH,1


In [2]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

def extract_DICOM_Part_16_table_data(xml_url, label):
    # Parse the XML content
    response = requests.get(xml_url)
    root = ET.fromstring(response.content)

    # Find the child node with the specified label
    selected_node = None
    for child in root:
        if child.attrib.get('label') == label:
            selected_node = child
            break

    # Get all tables from the selected node
    if selected_node is not None:
        table_elements = selected_node.findall('.//{http://docbook.org/ns/docbook}table')

        # Dictionary to store DataFrames
        tables_dict = {}

        # Iterate through each table
        for index, table_element in enumerate(table_elements):
            thead = table_element.find('.//{http://docbook.org/ns/docbook}thead')
            tbody = table_element.find('.//{http://docbook.org/ns/docbook}tbody')

            # Store column names
            column_names = []
            if thead is not None:
                # Store the column names for the first row
                for tr in thead.findall('.//{http://docbook.org/ns/docbook}tr'):
                    for idx in range(len(tr)):
                        if tr[idx][0].text is not None:
                            column_names.append(tr[idx][0].text.strip())

            # Store row values
            rows_data = []
            if tbody is not None:
                # Iterate through each row in tbody
                for tr in tbody.findall('.//{http://docbook.org/ns/docbook}tr'):
                    row_values = {}
                    for idx, td in enumerate(tr.findall('.//{http://docbook.org/ns/docbook}td')):
                        if idx < len(column_names):
                            # Extract values from nested elements, such as links
                            value = ' '.join(text.strip() for text in td.itertext())
                            if value:
                                row_values[column_names[idx]] = value.strip()

                    rows_data.append(row_values)

            # Save the output as a DataFrame and add to the dictionary
            df = pd.DataFrame(rows_data, columns=column_names)
            tables_dict[f'df_{index + 1}'] = df

        return tables_dict

    else:
        print(f"Node with label='{label}' not found.")
        return None

In [3]:
# URI for DICOM Standard Part 16
xml_uri = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part16/part16.xml'

# Extract tables with label 'B'
result_dict = extract_DICOM_Part_16_table_data(xml_uri, 'B')

In [85]:
result_dict['df_1101']

,,,,,
0,C0278604,C0278604,C0278604,C0278604,C0278604
1,C1306754,C1306754,C1306754,C1306754,C1306754
2,C0031831,C0031831,C0031831,C0031831,C0031831
3,C2985479,C2985479,C2985479,C2985479,C2985479
4,C1320928,C1320928,C1320928,C1320928,C1320928
5,C1441532,C1441532,C1441532,C1441532,C1441532
6,C2985483,C2985483,C2985483,C2985483,C2985483
7,Physics Assistant,Physics Assistant,Physics Assistant,Physics Assistant,Physics Assistant
8,C1708969,C1708969,C1708969,C1708969,C1708969


In [87]:
'''The Tables have 3 shapes
3 Cols = 
4 Cols
5 Cols = Coding Scheme Designator |
            Code Value	|
            Code Meaning |	
            SNOMED-RT ID |	
            UMLS Concept Unique ID
6 Cols = Coding Scheme Designator |
            Code Value	|
            Code Meaning |	
            SNOMED-RT ID |	
            UMLS Concept Unique ID |
            Trade Name (Informative)

Found bogus table 'df_759'
'''

faulty_tables=['df_759','df_901','df_1003','df_1100','df_1101']
table_names=list(result_dict.keys())
print("{} Tables Identified".format(len(table_names)))
print(table_names)
for faulty_table in faulty_tables:
    if faulty_table in table_names:
        print("Removing Faulty Table {}".format(faulty_table))
        table_names.remove(faulty_table)
df_list=[result_dict[x] for x in table_names]
col_list=[]
for item in df_list:
    col_list=col_list+item.columns.to_list()
df=pd.DataFrame(columns=list(set(col_list)))
df
ct=0
for table in df_list:
    print(ct)
    ct+=1
    df.merge(table,how='outer')



    

1347 Tables Identified
['df_1', 'df_2', 'df_3', 'df_4', 'df_5', 'df_6', 'df_7', 'df_8', 'df_9', 'df_10', 'df_11', 'df_12', 'df_13', 'df_14', 'df_15', 'df_16', 'df_17', 'df_18', 'df_19', 'df_20', 'df_21', 'df_22', 'df_23', 'df_24', 'df_25', 'df_26', 'df_27', 'df_28', 'df_29', 'df_30', 'df_31', 'df_32', 'df_33', 'df_34', 'df_35', 'df_36', 'df_37', 'df_38', 'df_39', 'df_40', 'df_41', 'df_42', 'df_43', 'df_44', 'df_45', 'df_46', 'df_47', 'df_48', 'df_49', 'df_50', 'df_51', 'df_52', 'df_53', 'df_54', 'df_55', 'df_56', 'df_57', 'df_58', 'df_59', 'df_60', 'df_61', 'df_62', 'df_63', 'df_64', 'df_65', 'df_66', 'df_67', 'df_68', 'df_69', 'df_70', 'df_71', 'df_72', 'df_73', 'df_74', 'df_75', 'df_76', 'df_77', 'df_78', 'df_79', 'df_80', 'df_81', 'df_82', 'df_83', 'df_84', 'df_85', 'df_86', 'df_87', 'df_88', 'df_89', 'df_90', 'df_91', 'df_92', 'df_93', 'df_94', 'df_95', 'df_96', 'df_97', 'df_98', 'df_99', 'df_100', 'df_101', 'df_102', 'df_103', 'df_104', 'df_105', 'df_106', 'df_107', 'df_108', 'df_

MergeError: Data columns not unique: Index([''], dtype='object')

In [23]:
result_dict['df_5'].head(20)

,Coding Scheme Designator,Code Value,Code Meaning,SNOMED-RT ID,UMLS Concept Unique ID
0,SCT,103381007,Trans-hepatic,G-D027,C0522516
1,SCT,103353001,Trans-gastric,G-A1B2,C0442355
2,SCT,103348006,Trans-pleural,G-A1A5,C0522494
3,SCT,103354007,Trans-mural,G-A1B3,C0522497
4,SCT,129226004,Trans-orbital,G-D065,C0442367
5,SCT,103349003,Trans-pancreatic,G-A1A6,C0522495
6,SCT,6706003,Trans-rectal,G-D028,C0205518
7,SCT,103347001,Trans-renal,G-A1A4,C0522493
8,SCT,103382000,Trans-temporal,G-D032,C0522517
9,SCT,103345009,Trans-thecal,G-A1A2,C0522492
